In [ ]:
# !pip install -U torch torchvision
# !pip install pdf2image
# !sudo apt install poppler-utils
# !pip install git+https://github.com/facebookresearch/fvcore.git
# import torch, torchvision
# torch.__version__
# !git clone https://github.com/facebookresearch/detectron2 detectron2_repo
# !pip install -e detectron2_repo

# Training

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("table", {}, "/content/drive/MyDrive/Fintab/Object Detection/Fintab_val_annot.json", "/content/drive/MyDrive/Fintab/data/FinTabNet_data")

In [ ]:
table_metadata = MetadataCatalog.get("table")
dataset_dicts = DatasetCatalog.get("table")

[12/03 17:43:00 d2.data.datasets.coco]: Loaded 5943 images in COCO format from /content/drive/MyDrive/Fintab/Object Detection/Fintab_val_annot.json


In [ ]:
len(table_metadata.thing_classes)

1

In [ ]:
import random

for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=table_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    print(vis)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
!wget https://layoutlm.blob.core.windows.net/tablebank/model_zoo/detection/All_X152/All_X152.yaml -O TB_X152.yaml
!wget https://layoutlm.blob.core.windows.net/tablebank/model_zoo/detection/All_X152/model_final.pth -O TB_model_X152.pth

--2021-12-03 17:37:07--  https://layoutlm.blob.core.windows.net/tablebank/model_zoo/detection/All_X152/All_X152.yaml
Resolving layoutlm.blob.core.windows.net (layoutlm.blob.core.windows.net)... 20.150.78.68
Connecting to layoutlm.blob.core.windows.net (layoutlm.blob.core.windows.net)|20.150.78.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 536 [text/plain]
Saving to: ‘TB_X152.yaml’

TB_X152.yaml        100%[===================>]     536  --.-KB/s    in 0s      

2021-12-03 17:37:08 (98.9 MB/s) - ‘TB_X152.yaml’ saved [536/536]

--2021-12-03 17:37:08--  https://layoutlm.blob.core.windows.net/tablebank/model_zoo/detection/All_X152/model_final.pth
Resolving layoutlm.blob.core.windows.net (layoutlm.blob.core.windows.net)... 20.150.78.68
Connecting to layoutlm.blob.core.windows.net (layoutlm.blob.core.windows.net)|20.150.78.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1103832675 (1.0G) [application/octet-stream]
Saving to: ‘TB_mod

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
# cfg.merge_from_file("./detectron2_repo/configs/COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.merge_from_file("/content/TB_X152.yaml")
# cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("table",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "/content/TB_model_X152.pth"
# cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x/138205316/model_final_a3ec72.pkl"
# cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(table_metadata.thing_classes) # 2 classes (red label, lipton yellow label)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/03 17:43:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[12/03 17:43:27 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/03 17:44:04 d2.utils.events]:  eta: 0:28:57  iter: 19  total_loss: 0.9862  loss_cls: 0.5675  loss_box_reg: 0.2846  loss_rpn_cls: 0.03288  loss_rpn_loc: 0.008697  time: 1.8160  data_time: 0.0674  lr: 1.9981e-06  max_mem: 7201M
[12/03 17:44:39 d2.utils.events]:  eta: 0:28:20  iter: 39  total_loss: 0.5523  loss_cls: 0.2105  loss_box_reg: 0.1954  loss_rpn_cls: 0.005133  loss_rpn_loc: 0.0087  time: 1.7963  data_time: 0.0049  lr: 3.9961e-06  max_mem: 7201M
[12/03 17:45:19 d2.utils.events]:  eta: 0:27:47  iter: 59  total_loss: 0.4725  loss_cls: 0.2369  loss_box_reg: 0.251  loss_rpn_cls: 0.005069  loss_rpn_loc: 0.005078  time: 1.8530  data_time: 0.0049  lr: 5.9941e-06  max_mem: 7689M
[12/03 17:45:54 d2.utils.events]:  eta: 0:27:11  iter: 79  total_loss: 0.5706  loss_cls: 0.1847  loss_box_reg: 0.258  loss_rpn_cls: 0.00475  loss_rpn_loc: 0.008753  time: 1.8295  data_time: 0.0052  lr: 7.9921e-06  max_mem: 7689M
[12/03 17:46:25 d2.utils.events]:  eta: 0:26:34  iter: 99  total_loss: 0.5221  los

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "TB_FT_model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.DATASETS.TEST = ("table")        
predictor = DefaultPredictor(cfg)

In [ ]:
!cp "/content/output/X152/All_X152/TB_FT_model_final.pth" "/content/drive/MyDrive/Fintab/Object_Detection"

In [ ]:
from detectron2.utils.visualizer import ColorMode
import shutil
import glob
# #im = "shutterstock_519484543.jpg"
# # for path in glob.glob('/content/*.jpg'):
# # im = cv2.imread(path)
# im = cv2.imread("/content/20Dec_Page_10.jpeg")
# # im = cv2.resize(im,(1024,1024))

# outputs = predictor(im)
# v = Visualizer(im[:, :, ::-1],
#                   metadata=table_metadata, 
#                   scale=0.8, 
#                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
# )
# v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# # plt.figure(figsize=(8,8))
# cv2_imshow(v.get_image()[:, :, ::-1])

# print(len(outputs["instances"]))
i=0
for image in os.listdir('/content/drive/MyDrive/Fintab/Object Detection/Testing data'):
  im = cv2.imread(os.path.join("/content/drive/MyDrive/Fintab/Object Detection/Testing data",image))
  print(image)
  i+=1
  if i==100:
    break
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],metadata=table_metadata,scale=0.8,instance_mode=ColorMode.IMAGE_BW)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  image_tobeshown=v.get_image()[:, :, ::-1]
  # cv2_imshow(image_tobeshown)
  plt.figure(figsize=(15,15))
  plt.imshow(v.get_image()[:, :, ::-1])
  plt.show()
  # image_name="inferenced_"+image.split('.')[0]+".jpg"
  # cv2.imwrite(image_name,image_tobeshown)
  # shutil.copy(image_name,"/content/drive/MyDrive/images/ananta/inferenced/") 

In [ ]:
import time
times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

Average(sec):0.11,fps:9.22


# Inference

In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from PIL import Image
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
import csv
from collections import namedtuple
import os
import io
import tempfile
import random
import tensorflow as tf
import numpy as np
import base64
from google.protobuf import text_format
from bs4 import BeautifulSoup
from pdf2image import convert_from_path
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.image as mpimg
from IPython.display import HTML
import urllib.request

try:
    from PIL import Image
except ImportError:
    import Image

Populating the interactive namespace from numpy and matplotlib


In [2]:
!wget https://layoutlm.blob.core.windows.net/tablebank/model_zoo/detection/All_X152/All_X152.yaml -O TB_X152.yaml
# !wget https://layoutlm.blob.core.windows.net/tablebank/model_zoo/detection/All_X152/model_final.pth -O TB_model_X152.pth

--2021-12-13 10:16:13--  https://layoutlm.blob.core.windows.net/tablebank/model_zoo/detection/All_X152/All_X152.yaml
Resolving layoutlm.blob.core.windows.net (layoutlm.blob.core.windows.net)... 20.150.78.68
Connecting to layoutlm.blob.core.windows.net (layoutlm.blob.core.windows.net)|20.150.78.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 536 [text/plain]
Saving to: ‘TB_X152.yaml’

TB_X152.yaml        100%[===================>]     536  --.-KB/s    in 0s      

2021-12-13 10:16:14 (113 MB/s) - ‘TB_X152.yaml’ saved [536/536]



In [3]:
# !cp '/content/TB_X152.yaml' '/content/drive/MyDrive/Fintab/Object Detection'
# !cp '/content/TB_model_X152.pth' '/content/drive/MyDrive/Fintab/Object Detection'

In [23]:
cfg = get_cfg()
cfg.merge_from_file("/content/TB_X152.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4  # set threshold for this model
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Fintab/Object Detection/TB_model_X152.pth"
# cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [4]:
def detect_table(image64):
  
  im = Image.open(io.BytesIO(base64.b64decode(image64)))
  # im = im.resize((768,768))
  im = np.array(im)
  start_time = time.time()
  outputs = predictor(im)
  delta = time.time() - start_time
  print(delta)
  if len(outputs["instances"])>0:
    return True
  else:
    return False

In [19]:
from pdf2image import convert_from_path
import pandas as pd
from detectron2.utils.visualizer import ColorMode
import shutil
import glob
import os

df = pd.read_csv('/content/drive/MyDrive/Fintab/Test PDF Links.csv')
image_array={'image_name':[],'im64':[]}
for i,row in enumerate(df.iterrows()):
  
  url = row[1]['url']
  pdf_path = url.replace(' ','%20')
  if 'Quarterly' not in pdf_path:
    continue

  print(f"Readin file {pdf_path}")
  # try:
  urllib.request.urlretrieve(pdf_path, "filename.pdf")
  pages = convert_from_path('filename.pdf',200)

  page_table = {}
  for j,page in enumerate(pages):
    im = Image.fromarray(np.array(page))
    im_file = io.BytesIO()
    im.save(im_file, format="JPEG")
    im_bytes = im_file.getvalue()  # im_bytes: image in binary format.
    im_b64 = base64.b64encode(im_bytes)
    image_array['image_name'].append(f'image_{i}_{j}')
    image_array['im64'].append(im_b64)
    print(detect_table(im_b64))

    # im = np.array(page)
    # im = np.array(page)
    # start_time = time.time()
    # outputs = predictor(im)
    # delta = time.time() - start_time
    # print(delta)
    # if len(outputs["instances"])>0:
    #   page_table[j+1] = len(outputs["instances"])
    # v = Visualizer(im[:, :, ::-1],scale=0.8,instance_mode=ColorMode.SEGMENTATION)
    # v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # image_tobeshown=v.get_image()[:, :, ::-1]
    # plt.imshow(v.get_image()[:, :, ::-1])
    # plt.show()

    
  # except:    
  #   print('error reading file')

  break

In [ ]:
from detectron2.utils.visualizer import ColorMode
import shutil
import glob
import os
# #im = "shutterstock_519484543.jpg"
# # for path in glob.glob('/content/*.jpg'):
# # im = cv2.imread(path)
# im = cv2.imread("/content/20Dec_Page_10.jpeg")
# # im = cv2.resize(im,(1024,1024))

# outputs = predictor(im)
# v = Visualizer(im[:, :, ::-1],
#                   metadata=table_metadata, 
#                   scale=0.8, 
#                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
# )
# v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# # plt.figure(figsize=(8,8))
# cv2_imshow(v.get_image()[:, :, ::-1])

# print(len(outputs["instances"]))
# sum=0
# tot=0
# ------------------------------
# dic={}
# for image in glob.glob('/content/drive/MyDrive/Fintab/Object Detection/Testing data/*.jpeg'):
#   # if tot==100:
#   #   break
#   if image.split('_')[1] in [24,25,26]:
#     im = cv2.imread(image)
#     # print(image)
#     outputs = predictor(im)
#     # tot+=1
#     # print(len(outputs['instances']))
#     if len(outputs['instances'])>=1:
#       sum+=1
#     # print(outputs['instances'])
#     v = Visualizer(im[:, :, ::-1],scale=0.8,instance_mode=ColorMode.SEGMENTATION)  
#     v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#     image_tobeshown=v.get_image()[:, :, ::-1]
#     # cv2_imshow(image_tobeshown)
#     # plt.figure(figsize=(10,10))
#     # plt.imshow(v.get_image()[:, :, ::-1])
#     # plt.show()
#     image_name="inferenced_"+image.split('/')[-1]
#     cv2.imwrite(image_name,image_tobeshown)
#     shutil.copy(image_name,"/content/drive/MyDrive/Fintab/Object Detection/Testing data/Predicted/") 

from pdf2image import convert_from_path
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Fintab/Test PDF Links.csv')
page_array = []
page_count = []
for i,row in enumerate(df.iterrows()):
  if i!=4:
    continue
  url = row[1]['url']
  pdf_path = url.replace(' ','%20')
  if 'Quarterly' not in pdf_path:
    continue

  print(f"Readin file {pdf_path}")
  try:
    urllib.request.urlretrieve(pdf_path, "filename.pdf")
    pages = convert_from_path('filename.pdf',200)

    page_table = {}
    for j,page in enumerate(pages):
      # page.save(f'/content/drive/MyDrive/Fintab/Object Detection/Testing data/image_{i}_{j}.jpeg', 'JPEG')
      path = f'/content/drive/MyDrive/Fintab/Object Detection/Testing data/image__special_special_{i}_{j}.jpeg'
      # im = cv2.imread(path)
      # # im = cv2.imread("/content/20Dec_Page_10.jpeg")
      # # im = cv2.resize(im,(1024,1024))

      im = np.array(page)
      im = cv2.resize(im,(1024,1024))
      start_time = time.time()
      outputs = predictor(im)
      delta = time.time() - start_time
      print(delta)
      # if len(outputs["instances"])>0:
      #   page_table[j+1] = len(outputs["instances"])
      v = Visualizer(im[:, :, ::-1],scale=0.8,instance_mode=ColorMode.SEGMENTATION)
      v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      # plt.figure(figsize=(8,8))
      image_tobeshown=v.get_image()[:, :, ::-1]
      # cv2_imshow(v.get_image()[:, :, ::-1])
      image_name="inferenced_"+path.split('/')[-1]
      cv2.imwrite(image_name,image_tobeshown)
      shutil.copy(image_name,"/content/drive/MyDrive/Fintab/Object Detection/Testing data/Predicted Quarterly/")

    # pdf = PdfFileReader(open('filename.pdf','rb'))
    # pages = pdf.getNumPages()
    
    # for i in range(1,pages+1):
    #   dfs = tabula.read_pdf(pdf_path, stream=True,pages=str(i))
    #   # read_pdf returns list of DataFrames
    #   if len(dfs)!=0:
    #     page_table[i]=len(dfs)
    print('Saving page array and page_count')
    page_array.append(list(page_table.keys()))
    page_count.append(len(page_table))
  except:
    page_array.append([])
    page_count.append(0)
    print('error reading file')

  # break

In [ ]:
dfa = df.iloc[0:19]
dfa['page_array'] = page_array
dfa['page_count'] = page_count
dfa.to_csv('Test Save.csv',index=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
!cp '/content/Test Save.csv' '/content/drive/MyDrive/Fintab/Object Detection/Testing data/Predicted Quarterly/'

In [14]:
import time
times = []
for i in range(5):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

Average(sec):6.00,fps:0.17
